### Create a Python Virtual Environment and Install requirements

In [1]:
!python -m venv ~/venvs/face_recognition

In [2]:
!source ~/venvs/face_recognition/bin/activate

In [ ]:
!pip install -r requirements.txt

### Folder Structure Creation

In [ ]:
import cv2
import os
import numpy as np

# ================= CONFIGURATION =================
person_name = input("Enter person name: ")
save_dir = os.path.join("dataset-raw", person_name)
os.makedirs(save_dir, exist_ok=True)

max_images = 50       # Number of images to capture
frame_interval = 10    # Capture every N frames
conf_threshold = 0.5  # Minimum confidence for face detection
# ================================================

# Load OpenCV DNN face detector
model_file = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
config_file = "deploy.prototxt"
if not os.path.exists(model_file) or not os.path.exists(config_file):
    raise FileNotFoundError("Make sure deploy.prototxt and caffemodel are in the working directory.")

net = cv2.dnn.readNetFromCaffe(config_file, model_file)

def detect_faces(frame, conf_threshold=0.5):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)
            faces.append((x1, y1, x2-x1, y2-y1))
    return faces

# Open webcam
cap = cv2.VideoCapture(0)
count = 0
frame_count = 0

print("Press 'q' to quit anytime.")

while count < max_images:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    faces = detect_faces(frame, conf_threshold)
    for (x, y, w, h) in faces:
        if frame_count % frame_interval == 0:
            face_crop = frame[y:y+h, x:x+w]
            filename = os.path.join(save_dir, f"{count+1}.jpg")
            cv2.imwrite(filename, face_crop)
            count += 1
            print(f"Saved: {filename}")
        # Draw rectangle for visualization
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Capture Dataset", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print(f"✅ Capture complete! {count} images saved for {person_name}.")


Enter person name:  Laurentti


Press 'q' to quit anytime.


### Auto Labeling

In [1]:
import cv2
import os
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


base_dir = os.getcwd() 
annotations_folder = os.path.join(base_dir, "annotations")

if not os.path.exists(annotations_folder):
    os.makedirs(annotations_folder)

class OpenCVDNNFaceDetector:
    def __init__(self, confidence_threshold=0.5):
        """
        Initialize OpenCV DNN Face Detector with pre-trained model
        """
        self.confidence_threshold = confidence_threshold
        
        # Model files
        self.model_file = "res10_300x300_ssd_iter_140000_fp16.caffemodel"
        self.config_file = "deploy.prototxt"
        
        # Verify files exist
        if not os.path.exists(self.model_file):
            available_models = [f for f in os.listdir('.') if f.endswith('.caffemodel')]
            raise FileNotFoundError(f"Model file not found. Available: {available_models}")
        if not os.path.exists(self.config_file):
            available_configs = [f for f in os.listdir('.') if 'deploy' in f and '.prototxt' in f]
            raise FileNotFoundError(f"Config file not found. Available: {available_configs}")
        
        print("✅ Model files found, loading face detector...")
        
        # Load the model
        self.net = cv2.dnn.readNetFromCaffe(self.config_file, self.model_file)
        print("✅ Face detector loaded successfully!")
    
    def detect_faces(self, image_path):
        """Detect faces using OpenCV DNN"""
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"⚠️  Could not read image: {image_path}")
            return [], (0, 0)
        
        (h, w) = image.shape[:2]
        
        # Create blob from image
        blob = cv2.dnn.blobFromImage(
            cv2.resize(image, (300, 300)), 1.0, (300, 300),
            (104.0, 177.0, 123.0)
        )
        
        # Pass blob through network
        self.net.setInput(blob)
        detections = self.net.forward()
        
        faces = []
        
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            
            if confidence > self.confidence_threshold:
                # Extract bounding box coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                
                # Ensure coordinates are within image boundaries
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(w, x2)
                y2 = min(h, y2)
                
                width = x2 - x1
                height = y2 - y1
                
                # Only include valid detections
                if width > 0 and height > 0:
                    faces.append({
                        'bbox': (x1, y1, width, height),
                        'confidence': confidence
                    })
        
        return faces, image.shape

def annotate_dataset_opencv_dnn(dataset_folder, output_format='both', min_confidence=0.5):
    """
    Automatically annotate faces using OpenCV DNN
    """
    detector = OpenCVDNNFaceDetector(confidence_threshold=min_confidence)
    annotations = {}
    stats = {
        'total_images': 0,
        'images_with_faces': 0,
        'total_faces': 0,
        'failed_detections': 0
    }
    
    print("🚀 Starting automatic face annotation with OpenCV DNN...")
    print("=" * 60)
    
    for person_folder in sorted(os.listdir(dataset_folder)):
        person_path = os.path.join(dataset_folder, person_folder)
        
        if not os.path.isdir(person_path):
            continue
            
        print(f"\n🔍 Processing: {person_folder}")
        person_images = 0
        person_faces = 0
        
        for image_file in sorted(os.listdir(person_path)):
            if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(person_path, image_file)
                stats['total_images'] += 1
                person_images += 1
                
                # Detect faces with OpenCV DNN
                faces, image_shape = detector.detect_faces(image_path)
                
                # Filter by confidence
                confident_faces = [f for f in faces if f['confidence'] >= min_confidence]
                
                if len(confident_faces) > 0:
                    stats['images_with_faces'] += 1
                    person_faces += len(confident_faces)
                    stats['total_faces'] += len(confident_faces)
                    
                    # Store annotation
                    image_key = f"{person_folder}/{image_file}"
                    annotations[image_key] = {
                        'person': person_folder,
                        'image_path': image_path,
                        'image_size': {
                            'width': image_shape[1],
                            'height': image_shape[0],
                            'channels': image_shape[2] if len(image_shape) > 2 else 3
                        },
                        'faces': [],
                        'face_count': len(confident_faces),
                        'detection_confidence': min_confidence
                    }
                    
                    for i, face in enumerate(confident_faces):
                        x, y, w, h = face['bbox']
                        annotations[image_key]['faces'].append({
                            'face_id': i,
                            'bbox': {
                                'x': int(x),
                                'y': int(y),
                                'width': int(w),
                                'height': int(h)
                            },
                            'confidence': float(face['confidence']),
                            'bbox_normalized': {
                                'x': x / image_shape[1],
                                'y': y / image_shape[0],
                                'width': w / image_shape[1],
                                'height': h / image_shape[0]
                            }
                        })
                    
                    print(f"  ✅ {image_file}: {len(confident_faces)} face(s) detected")
                else:
                    stats['failed_detections'] += 1
                    print(f"  ⚠️  {image_file}: No faces detected")
        
        print(f"  📊 {person_folder}: {person_faces} faces in {person_images} images")
    
    # Print statistics
    print_stats(stats)
    
    # Export annotations
    output_files = []
    if output_format in ['json', 'both']:
        json_file = export_json_annotations(annotations, annotations_folder, stats)
        output_files.append(json_file)
    
    if output_format in ['xml', 'both']:
        xml_file = export_xml_annotations(annotations, annotations_folder, stats)
        output_files.append(xml_file)
    
    # Export individual XML files (PASCAL VOC format)
    if output_format in ['xml', 'both']:
        voc_folder = export_individual_xml_annotations(annotations, annotations_folder)
        output_files.append(voc_folder)
    
    return annotations, output_files

def print_stats(stats):
    """Print annotation statistics"""
    print("\n" + "=" * 60)
    print("📊 ANNOTATION STATISTICS")
    print("=" * 60)
    print(f"Total images processed: {stats['total_images']}")
    print(f"Images with faces: {stats['images_with_faces']}")
    print(f"Images without faces: {stats['failed_detections']}")
    print(f"Total faces detected: {stats['total_faces']}")
    
    if stats['total_images'] > 0:
        success_rate = (stats['images_with_faces'] / stats['total_images']) * 100
        print(f"Success rate: {success_rate:.1f}%")
    
    print("=" * 60)

def export_json_annotations(annotations, output_folder, stats):
    """Export annotations to JSON format"""
    output_data = {
        'metadata': {
            'dataset_name': 'Face Recognition Dataset',
            'annotation_tool': 'OpenCV DNN Face Detection',
            'total_images': stats['total_images'],
            'images_with_faces': stats['images_with_faces'],
            'total_faces': stats['total_faces'],
            'export_timestamp': str(np.datetime64('now'))
        },
        'annotations': annotations
    }
    
    output_path = os.path.join(annotations_folder, 'face_annotations.json')
    
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)
    
    print(f"📄 JSON annotations saved to: {output_path}")
    return output_path

def export_xml_annotations(annotations, output_folder, stats):
    """Export annotations to single XML file"""
    root = ET.Element("face_annotations")
    
    # Add metadata
    metadata = ET.SubElement(root, "metadata")
    ET.SubElement(metadata, "dataset_name").text = "Face Recognition Dataset"
    ET.SubElement(metadata, "annotation_tool").text = "OpenCV DNN Face Detection"
    ET.SubElement(metadata, "total_images").text = str(stats['total_images'])
    ET.SubElement(metadata, "images_with_faces").text = str(stats['images_with_faces'])
    ET.SubElement(metadata, "total_faces").text = str(stats['total_faces'])
    
    # Add annotations
    annotations_elem = ET.SubElement(root, "images")
    for image_key, annotation in annotations.items():
        image_elem = ET.SubElement(annotations_elem, "image")
        
        ET.SubElement(image_elem, "person").text = annotation['person']
        ET.SubElement(image_elem, "file_path").text = annotation['image_path']
        
        size_elem = ET.SubElement(image_elem, "size")
        ET.SubElement(size_elem, "width").text = str(annotation['image_size']['width'])
        ET.SubElement(size_elem, "height").text = str(annotation['image_size']['height'])
        
        faces_elem = ET.SubElement(image_elem, "faces")
        for face in annotation['faces']:
            face_elem = ET.SubElement(faces_elem, "face")
            ET.SubElement(face_elem, "id").text = str(face['face_id'])
            ET.SubElement(face_elem, "confidence").text = str(face['confidence'])
            
            bbox_elem = ET.SubElement(face_elem, "bndbox")
            ET.SubElement(bbox_elem, "xmin").text = str(face['bbox']['x'])
            ET.SubElement(bbox_elem, "ymin").text = str(face['bbox']['y'])
            ET.SubElement(bbox_elem, "xmax").text = str(face['bbox']['x'] + face['bbox']['width'])
            ET.SubElement(bbox_elem, "ymax").text = str(face['bbox']['y'] + face['bbox']['height'])
    
    # Pretty print XML
    xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
    
    output_path = os.path.join(annotations_folder, 'face_annotations.xml')
    with open(output_path, 'w') as f:
        f.write(xml_str)
    
    print(f"📄 XML annotations saved to: {output_path}")
    return output_path

def export_individual_xml_annotations(annotations, output_folder):
    """Export individual XML files for each image (PASCAL VOC format)"""
    annotations_folder = os.path.join(output_folder, "annotations")
    if not os.path.exists(annotations_folder):
        os.makedirs(annotations_folder)
    
    for image_key, annotation in annotations.items():
        root = ET.Element("annotation")
        
        # Folder and filename
        ET.SubElement(root, "folder").text = "dataset"
        ET.SubElement(root, "filename").text = os.path.basename(annotation['image_path'])
        ET.SubElement(root, "path").text = annotation['image_path']
        
        # Source
        source = ET.SubElement(root, "source")
        ET.SubElement(source, "database").text = "Custom Face Dataset"
        ET.SubElement(source, "annotation").text = "OpenCV DNN Face Detection"
        
        # Image size
        size = ET.SubElement(root, "size")
        ET.SubElement(size, "width").text = str(annotation['image_size']['width'])
        ET.SubElement(size, "height").text = str(annotation['image_size']['height'])
        ET.SubElement(size, "depth").text = "3"
        
        # Segmentation (not used, but required by some tools)
        ET.SubElement(root, "segmented").text = "0"
        
        # Each face as an object
        for face in annotation['faces']:
            obj = ET.SubElement(root, "object")
            ET.SubElement(obj, "name").text = annotation['person']
            ET.SubElement(obj, "pose").text = "Unspecified"
            ET.SubElement(obj, "truncated").text = "0"
            ET.SubElement(obj, "difficult").text = "0"
            ET.SubElement(obj, "confidence").text = str(face['confidence'])
            
            bndbox = ET.SubElement(obj, "bndbox")
            ET.SubElement(bndbox, "xmin").text = str(face['bbox']['x'])
            ET.SubElement(bndbox, "ymin").text = str(face['bbox']['y'])
            ET.SubElement(bndbox, "xmax").text = str(face['bbox']['x'] + face['bbox']['width'])
            ET.SubElement(bndbox, "ymax").text = str(face['bbox']['y'] + face['bbox']['height'])
        
        # Save individual XML
        xml_str = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
        xml_filename = os.path.splitext(os.path.basename(annotation['image_path']))[0] + ".xml"
        xml_path = os.path.join(annotations_folder, xml_filename)
        
        with open(xml_path, 'w') as f:
            f.write(xml_str)
    
    print(f"📄 Individual XML annotations saved to: {annotations_folder}")
    return annotations_folder

def main():
    """Main function to run the complete annotation pipeline"""
    dataset_folder = "dataset-raw"  # Your standardized dataset
    output_format = "both"  # 'json', 'xml', or 'both'
    min_confidence = 0.5  # Minimum confidence for face detection
    
    print("🎯 OpenCV DNN Face Annotation Pipeline")
    print("=" * 60)
    
    try:
        # Run annotation
        annotations, output_files = annotate_dataset_opencv_dnn(
            dataset_folder, 
            output_format=output_format,
            min_confidence=min_confidence
        )
        
        print("\n✅ Annotation completed successfully!")
        print("📁 Output files:")
        for file_path in output_files:
            print(f"   - {file_path}")
            
    except FileNotFoundError as e:
        print(f"❌ {e}")
        print("\n💡 Please rename your model files to:")
        print("   - deploy.prototxt")
        print("   - res10_300x300_ssd_iter_140000_fp16.caffemodel")

if __name__ == "__main__":
    main()

🎯 OpenCV DNN Face Annotation Pipeline
✅ Model files found, loading face detector...
✅ Face detector loaded successfully!
🚀 Starting automatic face annotation with OpenCV DNN...

🔍 Processing: Laurentti
  ✅ 1.jpg: 1 face(s) detected
  ✅ 10.jpg: 1 face(s) detected
  ✅ 11.jpg: 1 face(s) detected
  ✅ 12.jpg: 1 face(s) detected
  ✅ 13.jpg: 1 face(s) detected
  ✅ 14.jpg: 1 face(s) detected
  ✅ 15.jpg: 1 face(s) detected
  ✅ 16.jpg: 1 face(s) detected
  ✅ 17.jpg: 1 face(s) detected
  ✅ 18.jpg: 1 face(s) detected
  ✅ 19.jpg: 1 face(s) detected
  ✅ 2.jpg: 1 face(s) detected
  ✅ 20.jpg: 1 face(s) detected
  ✅ 21.jpg: 1 face(s) detected
  ✅ 22.jpg: 1 face(s) detected
  ✅ 23.jpg: 1 face(s) detected
  ✅ 24.jpg: 1 face(s) detected
  ✅ 25.jpg: 1 face(s) detected
  ✅ 26.jpg: 1 face(s) detected
  ✅ 27.jpg: 1 face(s) detected
  ✅ 28.jpg: 1 face(s) detected
  ✅ 29.jpg: 1 face(s) detected
  ✅ 3.jpg: 1 face(s) detected
  ✅ 30.jpg: 1 face(s) detected
  ✅ 31.jpg: 1 face(s) detected
  ✅ 32.jpg: 1 face(s) dete

### Standardize the Dataset

In [2]:
import os
import json
import cv2

# Paths
base_dir = os.getcwd()  # Base project directory
annotations_file = os.path.join(base_dir, "annotations", "face_annotations.json")
output_folder = os.path.join(base_dir, "dataset-standardized")

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load annotations
with open(annotations_file, 'r') as f:
    data = json.load(f)

annotations = data.get("annotations", {})

# Target size for standardization
TARGET_SIZE = (224, 224)  # Typical size for InsightFace / ArcFace

for image_key, annotation in annotations.items():
    person = annotation['person']
    person_folder = os.path.join(output_folder, person)
    if not os.path.exists(person_folder):
        os.makedirs(person_folder)

    image_path = annotation['image_path']
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️  Could not read {image_path}")
        continue

    for i, face in enumerate(annotation['faces']):
        x, y, w, h = face['bbox']['x'], face['bbox']['y'], face['bbox']['width'], face['bbox']['height']

        # Ensure bounding box is within image
        x, y = max(0, x), max(0, y)
        w, h = max(1, w), max(1, h)
        cropped_face = img[y:y+h, x:x+w]

        # Resize to target size
        standardized_face = cv2.resize(cropped_face, TARGET_SIZE)

        # Save
        save_name = f"{i+1}_{os.path.basename(image_path)}"
        save_path = os.path.join(person_folder, save_name)
        cv2.imwrite(save_path, standardized_face)
        print(f"✅ Saved {save_path}")


✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_1.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_10.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_11.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_12.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_13.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_14.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_15.jpg
✅ Saved /home/steve/Python/Emerging-Technologies-in-CpE/real_time_face_recognition/dataset-standardized/Laurentti/1_16.jpg
✅ Saved /home/ste

### Dataset Augementation

In [3]:
import os
import cv2
import numpy as np

# Paths
base_dir = os.getcwd()
dataset_folder = os.path.join(base_dir, "dataset-standardized")
augmented_folder = os.path.join(base_dir, "dataset-augmented")

if not os.path.exists(augmented_folder):
    os.makedirs(augmented_folder)

# Parameters
ROTATION_ANGLES = [-15, -10, -5, 5, 10, 15]  # degrees
BRIGHTNESS_FACTORS = [0.7, 1.0, 1.3]  # multiply pixel values
ZOOM_FACTORS = [0.9, 1.0, 1.1]  # crop & resize
FLIP_MODES = [0, 1, -1]  # 0=vertical, 1=horizontal, -1=both

def augment_image(img):
    aug_images = []
    h, w = img.shape[:2]

    # Rotate
    for angle in ROTATION_ANGLES:
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        rotated = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)
        aug_images.append(rotated)

    # Flip
    for mode in FLIP_MODES:
        flipped = cv2.flip(img, mode)
        aug_images.append(flipped)

    # Brightness
    for factor in BRIGHTNESS_FACTORS:
        bright = np.clip(img * factor, 0, 255).astype(np.uint8)
        aug_images.append(bright)

    # Zoom
    for factor in ZOOM_FACTORS:
        if factor == 1.0:
            aug_images.append(img)
            continue
        new_h, new_w = int(h*factor), int(w*factor)
        y1 = (h - new_h)//2
        x1 = (w - new_w)//2
        zoomed = img[y1:y1+new_h, x1:x1+new_w]
        zoomed = cv2.resize(zoomed, (w, h))
        aug_images.append(zoomed)

    return aug_images

# Iterate over standardized dataset
for person in sorted(os.listdir(dataset_folder)):
    person_path = os.path.join(dataset_folder, person)
    if not os.path.isdir(person_path):
        continue

    out_person_folder = os.path.join(augmented_folder, person)
    if not os.path.exists(out_person_folder):
        os.makedirs(out_person_folder)

    for img_file in sorted(os.listdir(person_path)):
        if not img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        img_path = os.path.join(person_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        aug_images = augment_image(img)
        for i, aug_img in enumerate(aug_images):
            save_name = f"{os.path.splitext(img_file)[0]}_aug{i+1}.jpg"
            save_path = os.path.join(out_person_folder, save_name)
            cv2.imwrite(save_path, aug_img)

print("🎉 Dataset augmentation completed!")


🎉 Dataset augmentation completed!


### Build Face Embeddings

In [4]:
import os
import cv2
import numpy as np
import pickle
from insightface.app import FaceAnalysis

# Paths
base_dir = os.getcwd()
dataset_folder = os.path.join(base_dir, "dataset-standardized")  # or "dataset-augmented"
output_file = os.path.join(base_dir, "face_database.pkl")

# Initialize InsightFace
app = FaceAnalysis(providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(320, 320))  # smaller detection window

# Dictionary to store embeddings
face_database = {}

print("🚀 Building face embeddings...")

for person in sorted(os.listdir(dataset_folder)):
    person_path = os.path.join(dataset_folder, person)
    if not os.path.isdir(person_path):
        continue

    embeddings_list = []

    for img_file in sorted(os.listdir(person_path)):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(person_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️  Could not read image: {img_path}")
            continue

        faces = app.get(img)
        if len(faces) == 0:
            print(f"⚠️  No face detected in {img_file}")
            continue

        # Take the first detected face (if multiple)
        embedding = faces[0].embedding
        embeddings_list.append(embedding)

    if len(embeddings_list) > 0:
        face_database[person] = np.stack(embeddings_list)
        print(f"✅ {person}: {len(embeddings_list)} embeddings")

# Save embeddings to file
with open(output_file, 'wb') as f:
    pickle.dump(face_database, f)

print(f"🎉 Face embeddings saved to: {output_file}")
print(f"Total persons: {len(face_database)}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det

In [6]:
import pickle

with open("face_database.pkl", "rb") as f:
    db = pickle.load(f)

print(db.keys())       # Should show your person(s)
print(len(db["Laurentti"]))  # Should show 50 embeddings


dict_keys(['Laurentti', 'Steve'])
50


### Train InsightFace model

In [7]:
import os
import cv2
import numpy as np
import pickle
from insightface.app import FaceAnalysis

# -----------------------------
# Configuration
# -----------------------------
DATASET_FOLDER = "dataset-standardized"  # Each subfolder = person
OUTPUT_FILE = "face_database.pkl"
DET_SIZE = (320, 320)  # Detection size (smaller -> more sensitive to small faces)

# -----------------------------
# Initialize InsightFace
# -----------------------------
app = FaceAnalysis(providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=DET_SIZE)

# -----------------------------
# Build face database
# -----------------------------
face_database = {}

for person_name in sorted(os.listdir(DATASET_FOLDER)):
    person_path = os.path.join(DATASET_FOLDER, person_name)
    if not os.path.isdir(person_path):
        continue

    embeddings = []
    print(f"🚀 Processing {person_name}...")

    for img_file in sorted(os.listdir(person_path)):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        
        img_path = os.path.join(person_path, img_file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️  Could not read {img_file}")
            continue

        faces = app.get(img)
        if not faces:
            print(f"⚠️  No face detected in {img_file}")
            continue
        
        # Take the first detected face embedding
        embeddings.append(faces[0].embedding)

    if embeddings:
        face_database[person_name] = np.array(embeddings)
        print(f"✅ {person_name}: {len(embeddings)} embeddings")
    else:
        print(f"⚠️  No valid embeddings for {person_name}")

# -----------------------------
# Save the database
# -----------------------------
with open(OUTPUT_FILE, "wb") as f:
    pickle.dump(face_database, f)

print(f"🎉 Face database saved to: {os.path.abspath(OUTPUT_FILE)}")
print(f"Total persons: {len(face_database)}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/steve/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det

### Run the Facial Recognition Display (For Linux)

In [21]:
import subprocess
import os

env = os.environ.copy()
env['QT_QPA_PLATFORM'] = 'xcb'
subprocess.Popen(["/home/steve/venvs/facerec/bin/python", "detect.py"], env=env)

<Popen: returncode: None args: ['/home/steve/venvs/facerec/bin/python', 'det...>

### Run the Facial Recognition Display (For Windows)

In [ ]:
!start cmd /k "python interactive.py"